# Credit Risk Classification

Credit risk poses a classification problem that’s inherently imbalanced. This is because healthy loans easily outnumber risky loans. In this project, we’ll use various techniques to train and evaluate models with imbalanced classes. We’ll use a dataset of historical lending activity from a peer-to-peer lending services company to build a model that can identify the creditworthiness of borrowers.

## Steps:

* Split of the Data into Training and Testing Sets

* Creation of a Logistic Regression Model with the Original Data

* Prediction of a Logistic Regression Model with Resampled Training Data.

* Comparison between the two models.

* Summary Report: Results are included in the README file, which in this case act as the Final Report (Overview, Results, and Summary)

In [23]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

import warnings
warnings.filterwarnings('ignore')

---

## Split the Data into Training and Testing Sets

### Step 1: Read the `lending_data.csv` data from the `Resources` folder into a Pandas DataFrame.

In [24]:
# Read the CSV file from the Resources folder into a Pandas DataFrame
df=pd.read_csv(Path('Resources/lending_data.csv'))

# Review the DataFrame
df

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,52800,0.431818,5,1,22800,0
1,8400.0,6.692,43600,0.311927,3,0,13600,0
2,9000.0,6.963,46100,0.349241,3,0,16100,0
3,10700.0,7.664,52700,0.430740,5,1,22700,0
4,10800.0,7.698,53000,0.433962,5,1,23000,0
...,...,...,...,...,...,...,...,...
77531,19100.0,11.261,86600,0.653580,12,2,56600,1
77532,17700.0,10.662,80900,0.629172,11,2,50900,1
77533,17600.0,10.595,80300,0.626401,11,2,50300,1
77534,16300.0,10.068,75300,0.601594,10,2,45300,1


### Step 2: Create the labels set (`y`)  from the “loan_status” column, and then create the features (`X`) DataFrame from the remaining columns.

In [3]:
# Separate the data into labels and features

# Separate the y variable, the labels
y=df['loan_status']

# Separate the X variable, the features
X=df.drop(columns='loan_status')

In [4]:
# Review the y variable Series
y.head()

0    0
1    0
2    0
3    0
4    0
Name: loan_status, dtype: int64

In [5]:
# Review the X variable DataFrame
X.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
0,10700.0,7.672,52800,0.431818,5,1,22800
1,8400.0,6.692,43600,0.311927,3,0,13600
2,9000.0,6.963,46100,0.349241,3,0,16100
3,10700.0,7.664,52700,0.430740,5,1,22700
4,10800.0,7.698,53000,0.433962,5,1,23000


### Step 3: Check the balance of the labels variable (`y`) by using the `value_counts` function.

In [6]:
# Check the balance of our target values
y.value_counts()

0    75036
1     2500
Name: loan_status, dtype: int64

In [7]:
y.shape

(77536,)

### Step 4: Split the data into training and testing datasets by using `train_test_split`.

In [30]:
# Split the data using train_test_split
# Assign a random_state of 1 to the function
X_train, X_test, y_train, y_test=train_test_split(X,y, random_state=1, test_size=0.25)

---

## Create a Logistic Regression Model with the Original Data

###  Step 1: Fit a logistic regression model by using the training data (`X_train` and `y_train`).

In [31]:
# Instantiate the Logistic Regression model with a random_state parameter of 1 to the model
model=LogisticRegression(random_state=1)

# Fit the model using training data
model.fit(X_train, y_train)

LogisticRegression(random_state=1)

### Step 2: Save the predictions on the testing data labels by using the testing feature data (`X_test`) and the fitted model.

In [32]:
# Make a prediction using the testing data
y_pred_test=model.predict(X_test)

### Step 3: Evaluate the model’s performance

* Calculate the accuracy score of the model.

* Generate the confusion matrix.

* Print the classification report.

In [33]:
# Print the balanced_accuracy score of the model
balanced_accuracy_score(y_test, y_pred_test)

0.9520479254722232

In [34]:
# Generate a confusion matrix for the model
confusion_matrix(y_test, y_pred_test)

array([[18663,   102],
       [   56,   563]])

In [35]:
# Confusion matrix considers all values 1s&0s
confusion_matrix(y_test, y_pred_test).sum()==len(y_pred_test)

True

In [36]:
# Print the classification report for the model
print(" \n             Classification Report with The Umbalaced Data Model")
print(classification_report_imbalanced(y_test, y_pred_test))

 
             Classification Report with The Umbalaced Data Model
                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      0.99      0.91      1.00      0.95      0.91     18765
          1       0.85      0.91      0.99      0.88      0.95      0.90       619

avg / total       0.99      0.99      0.91      0.99      0.95      0.91     19384



### Step 4: Answer the following question.

At this point, we can assess how well does the logistic regression model predict both the `0` (healthy loan) and `1` (high-risk loan) labels.

Healthy loans are predicted with a precision of 100%, which is optimal, and a recall of 100%, which is also optimal. High-risk loans has a smaller precision of 85%, and a recall of 91%, which we hope we can improve by dealing with the imbalance data.The f1 score, which balances prediction and recall is equal to 88%, which is only OK.

* The 91% recall means that out of the 635 defaulted loans, 578 (=0.91*635) this model correctly classified as hig-risk loans, and 57 were wrongly classified as healthy loans. This imply that 57 approved loans are high-risk, and have the cost associated to a default.
* The 85% precision with 578 true positives (from the previous calculation), means that 680 (=578/0.85) loans where the total amount classified as high-risk loans. Then, 102 (0.15*680) healthy loans were wrongly classified as high-risk loans. This imply the loss of the opportunity cost of 102 loans that were not approved, but would have been profitable if we have done it.


---

## Predict a Logistic Regression Model with Resampled Training Data

### Step 1: Use the `RandomOverSampler` module from the imbalanced-learn library to resample the data. 

In [37]:
# Import the RandomOverSampler module form imbalanced-learn
from imblearn.over_sampling import RandomOverSampler

# Instantiate the random oversampler model with a random_state parameter of 1 to the model
random_oversampler=RandomOverSampler(random_state=1)

# Fit the original training data to the random_oversampler model
X_resampled, y_resampled=random_oversampler.fit_resample(X_train, y_train)


In [38]:
# Count the distinct values of the resampled labels data. Of course they should match.
y_resampled.value_counts()

0    56271
1    56271
Name: loan_status, dtype: int64

### Step 2: Use the `LogisticRegression` classifier and the resampled data to fit the model and make predictions.

In [39]:
# Instantiate the Logistic Regression model with a random_state parameter of 1
model_oversampled=LogisticRegression(random_state=1)

# Fit the model using the resampled training data
model_oversampled.fit(X_resampled, y_resampled)

# Make a prediction using the testing data
y_pred_resampled=model_oversampled.predict(X_test)

### Step 3: Evaluate the model’s performance

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [40]:
# Print the balanced_accuracy score of the model 
balanced_accuracy_score(y_test, y_pred_resampled)

0.9936781215845847

In [41]:
# Generate a confusion matrix for the model
confusion_matrix(y_test, y_pred_resampled)

array([[18649,   116],
       [    4,   615]])

In [42]:
# Print the classification report for the model
print(" \n             Classification Report with Re-balaced Data Model")
print(classification_report_imbalanced(y_test, y_pred_resampled))

 
             Classification Report with Re-balaced Data Model
                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      0.99      0.99      1.00      0.99      0.99     18765
          1       0.84      0.99      0.99      0.91      0.99      0.99       619

avg / total       0.99      0.99      0.99      0.99      0.99      0.99     19384



### Step 4: Answer the following question

Let's see how well does the logistic regression model, fit with oversampled data, predict both the `0` (healthy loan) and `1` (high-risk loan) labels.



**High-risk loans**

The model does predicted significantly better the high-risk loans when including resampling. Particularly, there is an amazing improvement in the recall from 0.91 to 1.00! That means that all of the negative falses (declare healthy loan when they were a highly risky) now are correctly labeled as 1 (high-risk loan). This is awesome in terms of controling the cost associated to the default of loans. Without resampling, the bank would have to absorve the default cost of those 57 high-risk loans badly classified by the previous model. Fortunately, when applying the resample to control the imbalance, we see that that cost of default is totally prevented by the improved model.

The opportunity cost associated to the 102 loans wrongly classified as high-risk loans is still there, though. The resampling wasn't able to improve in recognizing that caracterization as we see the same 0.85 precision score in both models.

**Healthy loans**

Healthy loans are well classified in both models. Since it was a very good classification using original data, the addition of oversample did not improve significantly the classification.
